# Websrapping a Paginas de Supermercados en Linea Mexicanos

Utilizaremos varias herramientas
- bs4 (Beautiful Soup)
- request 
- selenium 
- msedge.selenium_tools


El flujo es el siguiente: 
1. Ingresamos a la página web con el url que vamos a construir -> en este caso esta predeterminado pero ustedes pueden crear una funcion para automatizarlo quiza leyendo los valores de una BD o un JSON .... 
2.  Ahora relizaremos el scrapping -> identificaremos 3 valores importantes que vamos a necesitar <b>Nombre</b> , <b>Precio</b> , <b>Url</b> donde dependera como fue el diseño de las mismas para poder extrar la informacion
3. Procesaremos la información para posteriormente agregarla a un diccionario , y finalmente serializarla con un JSON a alguna base de datos, en nuestro caso utilizaremos <b>MONGO DB</b>
4. El scrapping esta listo, lo que falta ahora es ¿Qué hacer con esta informacion?

### Webscrapping Aurrera

Aqui tenemos ademas una cheatsheet de como realizar las principales utilidades de <b>Selenium</b>

 ['url cheatsheets'](https://www.automatetheplanet.com/selenium-webdriver-java-cheat-sheet/)

In [2]:
# Url de Bodega Aurrera
link = "https://www.bodegaaurrera.com.mx/productos?Ntt=comida%20gato" 

In [3]:
#pip install bs4 requests selenium msedge.selenium_tools 

In [4]:
# obtener el html de la pagina web link
import requests
from bs4 import BeautifulSoup

page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

# imprimir el html de la pagina web link
print(soup.prettify())


<html>
 <head>
  <title>
   Access Denied
  </title>
 </head>
 <body>
  <h1>
   Access Denied
  </h1>
  You don't have permission to access "http://www.bodegaaurrera.com.mx/productos?" on this server.
  <p>
   Reference #18.af794017.1679451500.1376157b
  </p>
 </body>
</html>



MMM nos marca un error de permisos --> 

Intentaremos realizar la busqueda con un request, pero como es una página con una restriccion de navegador y ademas <b>dinamica</b> , entonces buscaremos con SELENIUM

Para esta sección y las demás que veamos esta importacion de librerias, las dejare marcadas por si solo necesitan ejecutar ese bloque de codigo

In [5]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from msedge.selenium_tools import EdgeOptions
from msedge.selenium_tools import Edge
import time
PATH = "./msedgedriver.exe"

options = EdgeOptions()
options.use_chromium = True
options.headless = True
options.add_experimental_option(
    "excludeSwitches", ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
driver = Edge(options=options)


C:\Users\Adolfo\AppData\Local\Temp\ipykernel_15180\2784977948.py:18: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


In [6]:
driver = webdriver.Edge(PATH)
driver.get(link)
html = ""
# esperar 6 segundos -> esto es para que cargue la pagina web
time.sleep(6)
# obtener el html de la pagina web link
html = driver.page_source
driver.quit()

#### Nota: Debido a que tienen mejores sistemas de seguridad anti scrapers , no pudimos realizar nuestro trabajo con Aurrera, bien ahi grupo Walmart

In [7]:
# contar el numero de divs que hay en el siguiente xpath //*[@id="scrollContainer"]/section/div[1]/div[3]/div[2]/div[1]/div
# divs = driver.find_element(by=By.XPATH, value='//*[@id="scrollContainer"]/section/div[1]/div[3]/div[2]/div[1]/div')

# datos = divs.find_elements(By.TAG_NAME, "div")

# limpieza = []
# productos = []
# for x in datos:
#    limpieza.append(x.get_attribute("innerHTML"))


In [8]:
# crear diccionario con los datos de los productos
# import re
# nombre_temporal = ""
# for x in limpieza:
#    soup = BeautifulSoup(x, 'html.parser')


#    if(soup.find('a') != None):

#        precio = (soup.find(attrs={"data-automation-id" : "sale-price"}))
#        if(precio != None):
#            precio = precio.text
#            nombre = (soup.find(attrs={"data-automation-id" : "product-name"}).text)

#            producto = {
#                "nombre": nombre,
#                "precio": precio,
#                "url": "https://www.bodegaaurrera.com.mx"+soup.find('a').get('href')
#            }

#            if nombre != nombre_temporal:
#                productos.append(producto)
#                nombre_temporal = nombre
# driver.quit()


## WebScrapping para Chedraui

In [9]:
# con el webdriver de edge, abrir la pagina web link
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from msedge.selenium_tools import EdgeOptions
from msedge.selenium_tools import Edge
import time
import datetime
PATH = "./msedgedriver.exe"


# Crearemos un diccionario con los productos que queremos buscar

comida_gatoC = {
    "nombre": "Comida para gato",
    "url": "https://www.chedraui.com.mx/gatos?_q=gatos&map=ft"
}

mayonesaC = {
    "nombre": "Mayonesa",
    "url": "https://www.chedraui.com.mx/mayonesa?_q=mayonesa&map=ft"
}

fabulosoC = {
    "nombre": "Fabuloso",
    "url": "https://www.chedraui.com.mx/fabuloso?_q=fabuloso&map=ft"
}


diccionarioC = {
    "comida_gato": comida_gatoC,
    "mayonesa": mayonesaC,
    "fabuloso": fabulosoC
}

driver = webdriver.Edge(PATH)
#importante definir el tamaño de la ventana a maximizar por el tema de el CSS responsive
driver.maximize_window()
pChedraui = []


KeyboardInterrupt: 

In [ ]:
# obtener los precios de los productos de la pagina web link
for key, value in diccionarioC.items():
    driver.get(value["url"])
    html = ""
    # esperar 6 segundos
    time.sleep(6)
    # obtener el html de la pagina web link
    html = driver.page_source

    productosChedraui = driver.find_element(By.ID, "gallery-layout-container")
    productosChedraui = productosChedraui.find_elements(By.TAG_NAME, "div")
    contador = 1

    nombre_complete = ""
    precio_complete = ""

    for x in productosChedraui:

        elementoPrecio = "/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div[3]/div/div/div/div[" + str(
            contador)+"]/section/a/article/div[6]/div/div/span/span"
        elementoNombre = "/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div[3]/div/div/div/div["+str(
            contador)+"]/section/a/article/div[5]/h3/span"
        elementoUrl = "/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div/div/div[4]/div/div[2]/div/div/div/div/div[3]/div/div/div/div["+str(
            contador)+"]/section/a"

        try:
            precio = x.find_element(By.XPATH, elementoPrecio).text
        except Exception as e:
            precio = ""

        try:
            nombre = x.find_element(By.XPATH, elementoNombre).text
        except:
            nombre = ""
        try:
            url = x.find_element(By.XPATH, elementoUrl).get_attribute("href")
        except:
            url = ""

        if (nombre != "" and nombre != None):
            # print("Nombre: "+ nombre)
            nombre_complete = nombre
        if (precio != "" and precio != None):
            # print("Precio: "+ precio)
            precio_complete = precio
        if (url != "" and url != None):
            # print("Url: "+ url)
            url_complete = url

        if (nombre_complete == "" or precio_complete == "" or url_complete == ""):
            continue

        producto = {
            "nombre": nombre_complete,
            "busqueda": value["nombre"],
            "precio": precio_complete,
            "url": url_complete,
            "tienda": "Chedraui",
            "fecha": datetime.datetime.now().strftime("%Y-%m-%d")

        }

        contador = contador + 1

        pChedraui.append(producto)
    print(pChedraui)
contador = 1
driver.quit()


[{'nombre': 'Purina Cat Chow Defenseplus Gatitos Sabor Pescado Carne y Leche 1.5kg', 'busqueda': 'Comida para gato', 'precio': '$128.00', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-defenseplus-gatitos-sabor-pescado-carne-y-leche-15kg-3074427/p', 'tienda': 'Chedraui', 'fecha': '2023-03-20'}, {'nombre': 'Purina Cat Chow Defense Plus para Gatos Adultos Sabor Carne 3kg', 'busqueda': 'Comida para gato', 'precio': '$277.00', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-defense-plus-para-gatos-adultos-sabor-carne-3kg-3709060/p', 'tienda': 'Chedraui', 'fecha': '2023-03-20'}, {'nombre': 'Purina Cat Chow Delimix con Defenseplus Gatos Adultos 1.5kg', 'busqueda': 'Comida para gato', 'precio': '$119.00', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-delimix-con-defenseplus-gatos-adultos-15kg-3258972/p', 'tienda': 'Chedraui', 'fecha': '2023-03-20'}, {'nombre': 'Purina Cat Chow Defensehydro Adultos Pescado Alimento Húmedo Sobre 85g', 'busqueda': 'Comida para gato', 'precio': '$1

In [ ]:
pChedraui

[{'nombre': 'Purina Cat Chow Defenseplus Gatitos Sabor Pescado Carne y Leche 1.5kg',
  'busqueda': 'Comida para gato',
  'precio': '$128.00',
  'url': 'https://www.chedraui.com.mx/purina-cat-chow-defenseplus-gatitos-sabor-pescado-carne-y-leche-15kg-3074427/p',
  'tienda': 'Chedraui',
  'fecha': '2023-03-20'},
 {'nombre': 'Purina Cat Chow Defense Plus para Gatos Adultos Sabor Carne 3kg',
  'busqueda': 'Comida para gato',
  'precio': '$277.00',
  'url': 'https://www.chedraui.com.mx/purina-cat-chow-defense-plus-para-gatos-adultos-sabor-carne-3kg-3709060/p',
  'tienda': 'Chedraui',
  'fecha': '2023-03-20'},
 {'nombre': 'Purina Cat Chow Delimix con Defenseplus Gatos Adultos 1.5kg',
  'busqueda': 'Comida para gato',
  'precio': '$119.00',
  'url': 'https://www.chedraui.com.mx/purina-cat-chow-delimix-con-defenseplus-gatos-adultos-15kg-3258972/p',
  'tienda': 'Chedraui',
  'fecha': '2023-03-20'},
 {'nombre': 'Purina Cat Chow Defensehydro Adultos Pescado Alimento Húmedo Sobre 85g',
  'busqueda'

In [ ]:
# guardar los datos de los productos en un archivo json
import json
with open('productosChedraui.json', 'w') as outfile:
    json.dump(pChedraui, outfile)


In [ ]:
# TODO: REALIZAR LA Busqueda tambien con Justo
# urlJusto = "https://justo.mx/search?q=Gatos"


### Webscraping Soriana

In [ ]:
options = EdgeOptions()
options.use_chromium = True
options.headless = True
options.add_experimental_option(
    "excludeSwitches", ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
driver = Edge(options=options)


driver = webdriver.Edge(PATH)
driver.maximize_window()

pSoriana = []

driver.get(
    "https://www.soriana.com/buscar?q=gatos&cid=&search-button=&page=2&cref=0&view=grid")
html = ""
# esperar 5 segundos
time.sleep(6)
# obtener el html de la pagina web link
html = driver.page_source
# iterar 20 veces para obtener los 20 productos de la pagina web
for x in range(20):
    datos = "/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[" + \
        str(x)+"]/div[1]/div/div[1]/a/img"

    url = "/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[" + \
        str(x)+"]/div[1]/div/div[1]/a"

    try:

        objeto = driver.find_element(By.XPATH, datos)

        url = driver.find_element(By.XPATH, url)

        texto_json = (objeto.get_attribute("data-cbt"))
        ulr = url.get_attribute("href")

        objeto_json = json.loads(texto_json)
        print(objeto_json)
        # Acceder a la lista de productos
        productos = objeto_json['ecommerce']['click']['products']

        # Imprimir los productos
        # print(productos)

        a = productos[0]
        nombre_producto = productos[0]['name']
        precio_producto = productos[0]['price']
        url_producto = ulr

        # print("Precio", a['price'])
        # print("Nombre", a['name'])
        # print("Url", ulr)

        producto = {
            "precio": a['price'],
            "busqueda": "Comida para gato",
            "nombre": a['name'],
            "url": ulr,
            "tienda": "Soriana",
            "fecha": datetime.datetime.now().strftime("%Y-%m-%d")
        }

        pSoriana.append(producto)

    except Exception as e:
        objeto = ""
        print(e)
time.sleep(2)
driver.quit()


C:\Users\Adolfo\AppData\Local\Temp\ipykernel_8048\2969844606.py:8: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[0]/div[1]/div/div[1]/a/img"}
  (Session info: MicrosoftEdge=111.0.1661.44)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[1]/div[1]/div/div[1]/a/img"}
  (Session info: MicrosoftEdge=111.0.1661.44)

{'event': 'showProductClick', 'eventTypes': 'click', 'ecommerce': {'click': {'actionField': {'list': 'Search Results'}, 'products': [{'category': 'Mascotas', 'name': 'Alimento para gato Whiskas esterlizado 1.5kg', 'price': 109, 'id': '11586908', 'variant': '', 'brand': 'Whiskas', 'position': 0}]}, 'currencyCode': 'MXN'}}
{'event': 'showProductClick', 'eventTypes': 'click', 'ecommerce': {'click': {'actionField': {'list': 'Search Results'}, 'products': [{'category': 'Mascotas', 'name': 'Alimento Para Gato Whiskas 3 Kg', 'price': 239, 'id': '115

En este caso especifico no haremos uso de un diccionario para iterar la información , debido justamente a que presentamos un error en nuestra ejecucion de un error con un forbidden :_c 

In [ ]:
options = EdgeOptions()
options.use_chromium = True
options.headless = True
options.add_experimental_option(
    "excludeSwitches", ['enable-automation'])
options.add_experimental_option('useAutomationExtension', False)
options.add_argument('--disable-blink-features=AutomationControlled')
driver = Edge(options=options)


driver = webdriver.Edge(PATH)
driver.maximize_window()


driver.get(
    "https://www.soriana.com/buscar?q=mayonesa&cid=&search-button=&page=2&cref=0&view=grid")
html = ""
# esperar 5 segundos
time.sleep(6)
# obtener el html de la pagina web link
html = driver.page_source
# iterar 20 veces para obtener los 20 productos de la pagina web
for x in range(20):
    datos = "/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[" + \
        str(x)+"]/div[1]/div/div[1]/a/img"

    url = "/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[" + \
        str(x)+"]/div[1]/div/div[1]/a"

    try:

        objeto = driver.find_element(By.XPATH, datos)
        print(objeto)
        url = driver.find_element(By.XPATH, url)

        texto_json = (objeto.get_attribute("data-cbt"))
        ulr = url.get_attribute("href")

        objeto_json = json.loads(texto_json)

        # Acceder a la lista de productos
        productos = objeto_json['ecommerce']['click']['products']



        a = productos[0]
        nombre_producto = productos[0]['name']
        precio_producto = productos[0]['price']
        url_producto = ulr

        producto = {
            "precio": a['price'],
            "busqueda": "Mayonesa",
            "nombre": a['name'],
            "url": ulr,
            "tienda": "Soriana",
            "fecha": datetime.datetime.now().strftime("%Y-%m-%d")
        }

        pSoriana.append(producto)

    except Exception as e:
        objeto = ""
        print(e)
time.sleep(2)
driver.quit()


C:\Users\Adolfo\AppData\Local\Temp\ipykernel_8048\578507552.py:8: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(options=options)


Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[0]/div[1]/div/div[1]/a/img"}
  (Session info: MicrosoftEdge=111.0.1661.44)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[1]/div[1]/div/div[1]/a/img"}
  (Session info: MicrosoftEdge=111.0.1661.44)

<selenium.webdriver.remote.webelement.WebElement (session="ab1907b10c89505884b93a61f1b93872", element="2cbf7e4f-4f65-4531-bfeb-fa283d91cea3")>
<selenium.webdriver.remote.webelement.WebElement (session="ab1907b10c89505884b93a61f1b93872", element="7fdda9a3-35a5-4ef8-9032-3aac9bf7720d")>
<selenium.webdriver.remote.webelement.WebElement (session="ab1907b10c89505884b93a61f1b93872", element="db3add2e-dca0-4a2b-9c60-33a70a20edaa")>
<selenium.webdriver.remote.webelement.WebElement (session="ab1907b10c89505884b93a61f1b93872", element="eb0ddbb3-0263-4a

In [ ]:
pSoriana

[{'precio': 109,
  'busqueda': 'Comida para gato',
  'nombre': 'Alimento para gato Whiskas esterlizado 1.5kg',
  'url': 'https://www.soriana.com/alimento-para-gato-whiskas-esterlizado-1.5kg/11586908.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-20'},
 {'precio': 239,
  'busqueda': 'Comida para gato',
  'nombre': 'Alimento Para Gato Whiskas 3 Kg',
  'url': 'https://www.soriana.com/alimento-para-gato-whiskas-3-kg/11586931.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-20'},
 {'precio': 113,
  'busqueda': 'Comida para gato',
  'nombre': 'Alimento Para Gato Felix Sabor Atun',
  'url': 'https://www.soriana.com/alimento-para-gato-felix-sabor-atun/11590937.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-20'},
 {'precio': 95.5,
  'busqueda': 'Comida para gato',
  'nombre': 'Nucat Alimento Para Gato 1.8 Kilos',
  'url': 'https://www.soriana.com/nucat-alimento-para-gato-1.8-kilos/11254658.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-20'},
 {'precio': 103,
  'busqueda': 'Comida para

In [1]:
#importar la funcion exploracion del archivo chedrauiFabuloso.py para poder usarla
from exploraciones.chedrauiFabuloso import exploracion
exploracion()


Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[0]/div[1]/div/div[1]/a/img"}
  (Session info: MicrosoftEdge=111.0.1661.44)

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[3]/div/wainclude/div[1]/div[3]/div[2]/div/div[1]/div[1]/div/div[1]/a/img"}
  (Session info: MicrosoftEdge=111.0.1661.44)

<selenium.webdriver.remote.webelement.WebElement (session="f02254d32de98e45c9c6bbada5d37696", element="1bdddd2c-1971-4e67-8dae-a929f61e16cf")>
<selenium.webdriver.remote.webelement.WebElement (session="f02254d32de98e45c9c6bbada5d37696", element="0a333ac6-1b1a-4dfe-9549-a89e9c609808")>
<selenium.webdriver.remote.webelement.WebElement (session="f02254d32de98e45c9c6bbada5d37696", element="8d50dca2-ca63-408c-b365-039573bf9825")>
<selenium.webdriver.remote.webelement.WebElement (session="f02254d32de98e45c9c6bbada5d37696", element="4c4c10e1-edd2-48

[{'precio': 36.5,
  'busqueda': 'Mayonesa',
  'nombre': 'Limpiador Fabuloso Limon 1.65lt',
  'url': 'https://www.soriana.com/limpiador-fabuloso-limon-1.65lt/11480100.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-21'},
 {'precio': 35.5,
  'busqueda': 'Mayonesa',
  'nombre': 'Limpiador Líquido Fabuloso Antibacterial Mar Fresco 1 lt',
  'url': 'https://www.soriana.com/limpiador-liquido-fabuloso-antibacterial-mar-fresco-1-lt/371870.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-21'},
 {'precio': 20,
  'busqueda': 'Mayonesa',
  'nombre': 'Limpiador Líquido Fabuloso Trap Antibacterial Pera 828 ml',
  'url': 'https://www.soriana.com/limpiador-liquido-fabuloso-trap-antibacterial-pera-828-ml/11589003.html',
  'tienda': 'Soriana',
  'fecha': '2023-03-21'},
 {'precio': 34.5,
  'busqueda': 'Mayonesa',
  'nombre': 'Limpiador Líquido Fabuloso Eucalipto y Cítricos 828 ml',
  'url': 'https://www.soriana.com/limpiador-liquido-fabuloso-eucalipto-y-citricos-828-ml/11493147.html',
  'tienda': 'Sori

In [ ]:
# guardar los datos de los productos en un archivo json

import json
with open('productosSoriana.json', 'w') as outfile:
    json.dump(pSoriana, outfile)


----------------------


Podemos agregar una capa de limpieza de datos opcionalmente.... 

In [ ]:
#eliminar los duplicados de los productos de chedraui 
import pandas as pd
df = pd.read_json('productosChedraui.json')
df.drop_duplicates(subset ="nombre", keep = 'first', inplace = True)
df.to_json('productosChedraui.json', orient='records')


In [ ]:
#eliminar los duplicados de los productos de soriana
import pandas as pd
df = pd.read_json('productosSoriana.json')
df.drop_duplicates(subset ="nombre", keep = 'first', inplace = True)
df.to_json('productosSoriana.json', orient='records')


In [ ]:
#cambiar el formato de precio de los productos de chedraui de $100.00 a 100 
import json
with open('productosChedraui.json') as json_file:
    data = json.load(json_file)
    for p in data:
        precio = p['precio']
        precio = precio.replace("$", "")
        precio = precio.replace(",", "")
       
        p['precio'] = precio
with open('productosChedraui.json', 'w') as outfile:
    json.dump(data, outfile)

Ahora podemos ya almacenar en una base de datos las entradas --->

In [ ]:
# almacenar el json en una base de datos
import pymongo
from pymongo import MongoClient
import datetime
import json

# conectar a la base de datos
client = MongoClient('localhost', 27017)
db = client['productos']
collection = db['productos']


# insertar el json productosChedraui.json en la base de datos
with open('productosChedraui.json') as f:
    file_data = json.load(f)
collection.insert_many(file_data)

# insertar el json productosSoriana.json en la base de datos
with open('productosSoriana.json') as f:
    file_data = json.load(f)
collection.insert_many(file_data)


#collection.insert_many(pChedraui)
#collection.insert_many(pSoriana)


In [ ]:
# de productosChedraui.json y productosSoriana.json obtener los productos que esten en ambos json
import json
pChedraui = []
with open('productosChedraui.json') as f:
    file_data = json.load(f)
    for p in file_data:
        nombre = p['nombre']
        precio = p['precio']
        busqueda = p['busqueda']
        url = p['url']
        tienda = p['tienda']
        fecha = p['fecha']
        producto = {
            "precio": precio,
            "busqueda": busqueda,
            "nombre": nombre,
            "url": url,
            "tienda": tienda,
            "fecha": fecha
        }
        pChedraui.append(producto)


In [ ]:
#separar de nombre la marca y el gramaje con regular expresions

import re
for p in pChedraui:
    nombre = p['nombre']
    
    gramaje = re.findall(r'(\d+)', nombre)
    
    p['gramaje'] = gramaje[0]

    print(p)

#con spacy encontrar la marca de los productos 


{'precio': '128.00', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat Chow Defenseplus Gatitos Sabor Pescado Carne y Leche 1.5kg', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-defenseplus-gatitos-sabor-pescado-carne-y-leche-15kg-3074427/p', 'tienda': 'Chedraui', 'fecha': '2023-03-20', 'gramaje': '1'}
{'precio': '277.00', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat Chow Defense Plus para Gatos Adultos Sabor Carne 3kg', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-defense-plus-para-gatos-adultos-sabor-carne-3kg-3709060/p', 'tienda': 'Chedraui', 'fecha': '2023-03-20', 'gramaje': '3'}
{'precio': '119.00', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat Chow Delimix con Defenseplus Gatos Adultos 1.5kg', 'url': 'https://www.chedraui.com.mx/purina-cat-chow-delimix-con-defenseplus-gatos-adultos-15kg-3258972/p', 'tienda': 'Chedraui', 'fecha': '2023-03-20', 'gramaje': '1'}
{'precio': '11.80', 'busqueda': 'Comida para gato', 'nombre': 'Purina Cat Chow Defensehyd

In [ ]:
#python3 -m spacy download en_core_web_sm

In [ ]:
#!python3 -m spacy download es_core_news_sm

In [ ]:
!python3 --version

Python 3.10.9


In [ ]:
len(pChedraui)

14

In [ ]:
import spacy
nlp = spacy.load('es_core_news_sm')
for p in pChedraui:
    nombre = p['nombre']
    doc = nlp(nombre)
    for ent in doc.ents:
        if ent.label_ == 'PER' :
            p['marca'] = ent.text
            print(p['marca'])

Purina Cat Chow Defenseplus Gatitos Sabor Pescado Carne
Purina Cat Chow Defense Plus
Purina Cat Chow Delimix
Defenseplus Gatos Adultos
Purina Cat Chow Defensehydro Adultos Pescado
Alimento Húmedo
Purina Cat Chow Defensehydro Adultos Esterilizados Pollo
Purina Cat Chow Defensehydro Adultos Pollo Alimento Húmedo
Purina Cat Chow Defensehydro Adultos Carne Alimento Húmedo
Purina Cat Chow Defensehydro Adultos Esterilizados Carne
Mayonesa McCormick
Limón
Mayonesa McCormick
Mayonesa McCormick
Heinz Limón Doy Pack 340
Jugo de Limón
Mayonesa McCormick
